# Data loader

built with windsurf cascade

In [2]:
def parse_input(filename):
    with open(filename, 'r') as f:
        # Split the input into two parts based on double newlines
        parts = f.read().strip().split('\n\n')
        
        # Parse the grid
        grid = [list(line) for line in parts[0].split('\n')]
        
        # Parse the instructions - remove all newlines and keep as one string
        instructions = parts[1].replace('\n', '')
        
        return grid, instructions



# Part 1

Largely done by hand with infill support from windsurf

In [3]:
def find_start_position(grid):
    for y, row in enumerate(grid):
        for x, cell in enumerate(row):
            # print(cell)
            if cell == '@':
                return (x, y)
    return None

def move(pos, instruction):
    # print("starting move", pos, instruction)
    x, y = pos
    if grid[y][x] == '#':
        return False
    if grid[y][x] == '.':
        return True

    if instruction == '^':
        direction= (0, - 1)
    elif      instruction == 'v':

        direction= (0,  1)
    elif      instruction == '>':

        direction= ( 1, 0)
    elif      instruction == '<':

        direction = ( - 1, 0)
    else:
        raise ValueError(f"Invalid direction: {instruction}")
    if move((x + direction[0], y + direction[1]), instruction) == True:
        # print("Moving!!!")
        currentChar = grid[y][x]
        grid[y][x] = grid[y + direction[1]][x + direction[0]]
        grid[y + direction[1]][x + direction[0]] = currentChar
        return True
def score(grid):
    scores = []
    for y, row in enumerate(grid):
        for x, cell in enumerate(row):
            if cell == 'O' or cell == '[':
                scores.append(x + 100 * y)         
    return sum(scores)

In [12]:

grid, instructions = parse_input('15.txt')

for i in range(len(instructions)):
    # print(".", end='')
    move(find_start_position(grid), instructions[i])




# # Print the grid
# print("Grid:")
# for row in grid:
#     print(''.join(row))

print("Score:", score(grid))



Score: 1516281


# Part 2

Largely done by hand with infill support. Note the messiness of move2 as reflecting an evolving understanding based move2

In [5]:
def double_grid(grid):
    newrows = []
    for i in range(len(grid)):
        newrow = []
        for j in range(len(grid[0])):
            cell = grid[i][j]
            if cell == '#':
                newrow.extend(['#', '#'])
            elif cell == '.':
                newrow.extend(['.','.'])
            elif cell == 'O':
                newrow.extend(['[',']'])
            elif cell == '@':    
                newrow.extend(['@','.'])
        newrows.append((newrow))
    return newrows


grid, instructions = parse_input('15.test.txt')
grid = double_grid(grid)

for row in grid:
    print(''.join(row))

####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################


In [6]:
def move2(pos, instruction, test_only = False):
    # print("starting move", pos, instruction, "character:", grid[pos[1]][pos[0]])
    x, y = pos
    if grid[y][x] == '#':
        return False
    if grid[y][x] == '.':
        return True
    is_y = False
    if instruction == '^':
        direction= (0, - 1)
        is_y = True
    elif      instruction == 'v':
        direction= (0,  1)
        is_y = True
    elif      instruction == '>':

        direction= ( 1, 0)
    elif      instruction == '<':

        direction = ( - 1, 0)
    else:
        raise ValueError(f"Invalid direction: {instruction}")
    currentChar = grid[y][x]
    if is_y == True:
        # we need to ask what we are pushing against
        nextChar = grid[y + direction[1]][x + direction[0]]
        if nextChar == '[':
            if move2((x + 1, y+direction[1]), instruction, True):
                if move2((x, y+direction[1]), instruction, test_only):
                    move2((x + 1, y+direction[1]), instruction, test_only)
                    if test_only == False:
                        grid[y][x] = grid[y + direction[1]][x + direction[0]]
                        grid[y + direction[1]][x + direction[0]] = currentChar
                    return True
        elif nextChar == ']':
            if move2((x - 1, y+direction[1]), instruction, True):
                if move2((x, y+direction[1]), instruction, test_only):
                    move2((x - 1, y+direction[1]), instruction, test_only)
                    if test_only == False:
                        grid[y][x] = grid[y + direction[1]][x + direction[0]]
                        grid[y + direction[1]][x + direction[0]] = currentChar
                    return True
        else:
            if move2((x + direction[0], y + direction[1]), instruction, test_only): 
                if test_only == False:
                    grid[y][x] = grid[y + direction[1]][x + direction[0]]
                    grid[y + direction[1]][x + direction[0]] = currentChar
                return True
        return False


        test = True
        if currentChar == '[':
            test = move2((x + 1, y), instruction, True)
        elif currentChar == ']':
            
            test = move2((x - 1, y), instruction, True)
        if test == False:
            print("Returning false because this is not a valid move in an adjacent cell")
            return False
        print("I can move the adjacent thing in this direction", )
    if move2((x + direction[0], y + direction[1]), instruction, test_only) == True:
        if test_only == False:
            if is_y == True:
                if currentChar == '[':
                    move2((x - 1, y+direction[1]), instruction, test_only)
                    print("moving adjacent thing - left", instruction, test_only, currentChar)
                elif currentChar == ']':
                    move2((x + 1, y+direction[1]), instruction, test_only)
                    print("moving adjacent thing - right")
            # print("Moving!!!")
            grid[y][x] = grid[y + direction[1]][x + direction[0]]
            grid[y + direction[1]][x + direction[0]] = currentChar
        return True


    else: # not is_y - this is easy
        if move2((x + direction[0], y + direction[1]), instruction, test_only): 
            grid[y][x] = grid[y + direction[1]][x + direction[0]]
            grid[y + direction[1]][x + direction[0]] = currentChar
            return True
        return False


In [7]:
from IPython.display import clear_output
from time import sleep

grid, instructions = parse_input('15.txt')
grid = double_grid(grid)
# print("Starting positions")
# for row in grid:
#     print(''.join(row))

# print(find_start_position(grid))

for i in range(len(instructions)):
    move2(find_start_position(grid), instructions[i])


print("Score:", score(grid))

Score: 1527969


# Animated Visualizer
Built with Windsurf cascade and the input of Josiah! 

In [8]:
from PIL import Image, ImageDraw

def visualize_grid(grid, cell_size=30, colors=None):
    """
    Visualize a grid using Pillow.
    
    Args:
        grid (List[List[str]]): The grid to visualize
        cell_size (int): Size of each cell in pixels
        colors (dict): Optional color mapping for different cell types
        
    Returns:
        PIL.Image: The generated image
    """
    if colors is None:
        colors = {
            '#': '#999999',  # Wall - dark gray
            '.': '#000000',  # Empty space - white
            '@': '#FF0000',  # Player - red
            'O': '#0000FF',  # Object - blue
            '[': '#990033',  # Left bracket - blue
            ']': '#330099',  # Right bracket - blue
        }
    
    height = len(grid)
    width = len(grid[0])
    
    # Create a new image with a white background
    img = Image.new('RGB', (width * cell_size, height * cell_size), 'white')
    draw = ImageDraw.Draw(img)
    
    # Draw each cell
    for y in range(height):
        for x in range(width):
            cell = grid[y][x]
            color = colors.get(cell, '#CCCCCC')  # Default to gray for unknown cells
            
            # Calculate cell coordinates
            left = x * cell_size
            top = y * cell_size
            right = left + cell_size
            bottom = top + cell_size
            
            # Draw filled rectangle for the cell
            draw.rectangle([left, top, right, bottom], fill=color, outline='#000000')
            
            # Add a small border to make the grid visible
            draw.rectangle([left, top, right, bottom], outline='#000000')
    
    return img

def save_grid_image(grid, filename, cell_size=30, colors=None):
    """
    Create and save a visualization of the grid.
    
    Args:
        grid (List[List[str]]): The grid to visualize
        filename (str): Output filename (should end in .png)
        cell_size (int): Size of each cell in pixels
        colors (dict): Optional color mapping for different cell types
    """
    img = visualize_grid(grid, cell_size, colors)
    img.save(filename)
    return img


In [9]:
from IPython.display import clear_output
from time import sleep

grid, instructions = parse_input('15.txt')
grid = double_grid(grid)



for i in range(len(instructions)):
    # print("Move", i)
    # print(instructions[i], end='')
    # print('')
    move2(find_start_position(grid), instructions[i])
    img = visualize_grid(grid)
    display(img)
    sleep(0.05)
    clear_output(wait=True)
# for i in range(len(grid)):
#     print(''.join(grid[i]))
 display(img)
print("Score:", score(grid))



Score: 9021


In [10]:
 for i in range(len(grid)):
    print(''.join(grid[i]))

####################
##[].......[].[][]##
##[]...........[].##
##[]........[][][]##
##[]......[]....[]##
##..##......[]....##
##..[]............##
##..@......[].[][]##
##......[][]..[]..##
####################
